## The Hamiltonian for the spring-ball oscillating system given in paper https://arxiv.org/pdf/2303.13012.pdf

Following largely from section 3 of the paper

- $ H = -\begin{bmatrix} 0 & B \\ B^\dagger & 0 \end{bmatrix} $ where dim of $B$ is $N \text{x} M$, therefore $dim(H) = (N+M) \text{x} (N+M)$
- We pad $B$ with zeros such that $dim (B) = N^2 \text{x}N^2$ (according to appendix A1 and A4), therefore $dim(H) = 2N^2 \text{x} 2N^2$
- Now for $B$ we have, $BB^\dagger = A$, $ \begin{equation}
  \sqrt{M}B|j,k\rangle =
    \begin{cases}
      \sqrt{k_{jj}}|j\rangle & \text{if } j=k \\
      \sqrt{k_{jk}}(|j\rangle-|k\rangle) & \text{if } j<k\\
    \end{cases}       
\end{equation} $ and elements of $B^\dagger$ are either $\sqrt{k_{jk}/m_j}$ or $0$.

- $ A$ = $\sqrt{M}^{-1] F \sqrt{M}^{-1} $.

- $M$ is diagonal matrix of masses $(m_{jj}>0)$ and $F$ is the $N \text{x} N$ matrix whose diagonal and off-diagonal entries are $f_{jj} = \sum_k \kappa_{jk}$ and $f_{jk} = -\kappa_{jk}$,respectively. 

for the second point in $|j,k\rangle$ state is in vector [00,11,22,33,01,02,03,12,13,23]

#### Case : $$d = 1, E = 1, n = 2 \implies N = 4, \text{ no. of springs = 4 }, m_i = 1 \forall i \in [1,N], k_{i,j} \in [1,4], j>i, \forall (i,j) $$
$$\dot{\vec{x}}(0) = \begin{bmatrix} 1 \\ -1 \\ 0 \\  0 \end{bmatrix} \text{ and } \vec{x} = \begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \end{bmatrix}$$ 

In [1]:
import numpy as np
import scipy as sc
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import HGate
from qiskit import QuantumCircuit, qasm2
from qiskit.quantum_info import SparsePauliOp
from utils import Hamiltonian_Formulation
import pennylane as qml
from itertools import product
from classiq import (
    Output,
    QArray,
    QParam,
    List,
    QBit,
    exponentiation_with_depth_constraint,
    QNum,
    control,
    create_model,
    execute,
    prepare_state,
    qfunc,
    show,
    synthesize,
    H,
    Output,
    Z,
    allocate,
    qfunc,
    suzuki_trotter,
    PauliTerm,
    Pauli,
    QuantumProgram,
    write_qmod,
    Preferences,
    set_preferences,
    qdrift
)

from classiq import set_execution_preferences
from classiq.execution import ExecutionPreferences

import classiq
from qiskit import qasm3


# classiq.authenticate()

In [2]:
n = 2
N = 2**n
m = 1
E = 1
#mass matrix
M = np.diag(np.full(N,m)) 
#K matrix (spring constants)
k11 = 2
k12 = 1
k23 = 3
k34 = 2
K = np.array(([k11,k12,0,0],[k12,0,k23,0],[0,k23,0,k34],[0,0,k34,0]))


ham_formulation = Hamiltonian_Formulation(n, K, M)
ham_formulation.compute_Hamiltonian()
H_matrix = ham_formulation.H_matrix
H_matrix[1]
# print(len(H_matrix))

matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        , -0.        , -0.        , -0.        , -0.        ,
          1.        , -0.        , -0.        , -1.73205081, -0.        ,
         -0.        , -0.        , -0.        , -0.        , -0.        ,
         -0.        , -0.        ]])

## State Preparation

In [3]:
initial_state = np.array([1/np.sqrt(2),-1/np.sqrt(2)] + [0 for i in range(2*N**2-2)])
print(len(initial_state))
# use https://github.com/Qiskit/qiskit/issues/11735#issuecomment-1992145075 or Classiq

qc = QuantumCircuit(2*n+1)
qc.h(0)
qc.z(0)

init_state = Statevector.from_instruction(qc)

initial_state


32


array([ 0.70710678, -0.70710678,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

## Conditions for testing
- t = 5
- for n = 2, 5, 10
- accuracy = 99%
- note down accuracy, depth of circuit, number of gates (single and 2 qubit)

## Computing the final state classically

In [4]:
final_time=5

# exact_times = np.linspace(0, t, 101)

# We compute the exact evolution using the exp
final_state = init_state.evolve(sc.linalg.expm(-1j * final_time * H_matrix))
actual_state = qml.math.dm_from_state_vector(final_state.data)
print(final_state.data)

[-0.57759303+0.j          0.43432998+0.j          0.32556339+0.j
 -0.27313207+0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        -0.02243082j  0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.27112871j
  0.        +0.j          0.        +0.j          0.        -0.3132481j
  0.        +0.j          0.        +0.35357152j  0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j        ]


In [5]:
final_state.is_valid()

True

## Computing the Hamiltonian simulation on Classiq

### State preparation using Classiq

#### Obtaining the Sparse pauli op from qiskit

In [6]:
H_sparse_pauli = SparsePauliOp.from_operator(H_matrix)

### Converting the Hamiltonian into appropriate classiq form

In [7]:
paulis = []
coeffs = []
for i in range(len(H_sparse_pauli.paulis)):
    pauli = []
    coeffs.append(np.round(H_sparse_pauli.coeffs[i], 10).real)
    for ind_pauli in H_sparse_pauli.paulis[i]:
        match str(ind_pauli):
            case "I":
                pauli.append(Pauli.I)  
            case "X":
                pauli.append(Pauli.X) 
            case "Y":
                pauli.append(Pauli.Y) 
            case "Z":
                pauli.append(Pauli.Z) 
    paulis.append(pauli)

paulis[2], coeffs[4]

PauliTerm_list = []
for i in range(len(paulis)):
    PauliTerm_list.append(PauliTerm(pauli=paulis[i], coefficient=coeffs[i]))

PauliTerm_list[1]

struct_literal(PauliTerm, pauli=[Pauli.Z, Pauli.I, Pauli.I, Pauli.I, Pauli.X], coefficient=-0.0883883476)

## Trotter-Suzuki Simulation

In [78]:
# creating the Sparse Pauli Op
H_sparse_pauli = SparsePauliOp.from_operator(H_matrix)

# State preparation in Classiq
# importing the qmod file
    
@qfunc
def main(io: Output[QArray[QBit]]) -> None:
    allocate(2*n+1, io)
    H(io[0])
    Z(io[0])
    suzuki_trotter(
        PauliTerm_list,
        evolution_coefficient=final_time,
        order=6,
        repetitions=1,
        qbv=io,
    )



model = create_model(main)
qprog_suzuki = synthesize(model)
show(qprog_suzuki)

Opening: https://platform.classiq.io/circuit/2aa49729-b1d8-4a94-96c6-17577ecd8376?version=0.39.0


## Exponentiation Simulation

In [82]:
@qfunc
def main(io: Output[QArray[QBit]]) -> None:
    allocate(2*n+1, io)
    H(io[0])
    Z(io[0])
    exponentiation_with_depth_constraint(
        PauliTerm_list,
        evolution_coefficient=final_time,
        max_depth=3*10**4,
        qbv=io,
    )

model = create_model(main)
# write_qmod(model, "exponentiation")
qprog_exponentiation = synthesize(model)
show(qprog_exponentiation)

Opening: https://platform.classiq.io/circuit/08af3391-37e6-4ee1-ae16-ac693793ece7?version=0.39.0


## Qdrift Simulation

In [76]:
@qfunc
def main(io: Output[QArray[QBit]]) -> None:
    allocate(2*n+1, io)
    H(io[2*n])
    Z(io[2*n])
    qdrift(
        PauliTerm_list,
        evolution_coefficient=final_time,
        num_qdrift=100,
        qbv=io,
    )

preferences = Preferences(
    output_format=["qasm"], qasm3=True)
model = create_model(main)
model = set_preferences(model, preferences)
write_qmod(model, "qdrift")
qprog_qdrift = synthesize(model)
show(qprog_qdrift)

Opening: https://platform.classiq.io/circuit/dd6f11b0-ff1a-405d-bbf5-fc4df6eebe47?version=0.39.0


In [66]:
# Converting to qasm to get the results
qasm_prog_suzuki = QuantumProgram.from_qprog(qprog_suzuki).qasm
qasm_prog_exp = QuantumProgram.from_qprog(qprog_exponentiation).qasm
qasm_prog_qdrift = QuantumProgram.from_qprog(qprog_qdrift).qasm

In [67]:
final_qc = qasm3.loads(qasm_prog_exp)

## Comparisions in PennyLane (doesn't work because of some) 

In [68]:
Statevector.from_instruction(final_qc).data

array([-0.300767  -0.31187127j,  0.25936981+0.30786196j,
        0.00299646+0.00135447j, -0.00175121-0.02532987j,
        0.00157199+0.00480421j,  0.07373641-0.00855635j,
        0.00044738+0.0119319j , -0.00235744+0.00481644j,
        0.15843731-0.22973677j,  0.01411653+0.0012649j ,
        0.00323765-0.01121618j, -0.00123869-0.00320568j,
       -0.00724257+0.02405776j,  0.01777338-0.03231287j,
       -0.0029071 +0.01984245j, -0.0006701 -0.00353169j,
        0.2726311 -0.31241731j,  0.01848249+0.01324683j,
        0.005246  +0.00158523j, -0.37884941+0.26180912j,
       -0.00089041+0.00125196j,  0.01253992+0.05322387j,
        0.00245262-0.0127263j ,  0.00326578-0.00239188j,
       -0.36553061+0.02145324j,  0.00300341+0.00880313j,
        0.00360728-0.00943992j,  0.00659029-0.00556003j,
        0.00258505+0.00767036j, -0.18142076+0.08990994j,
       -0.00104797+0.00831595j,  0.00243163-0.00475988j])

In [69]:
obtained_state = qml.math.dm_from_state_vector(Statevector.from_instruction(final_qc).data)

In [70]:
qml.math.fidelity(obtained_state, actual_state)

0.19669544309626102